In [225]:
import re
import json
import datetime

import pandas as pd
import numpy as np

## Let's explore de provided dataset

In [217]:
data = pd.read_json('data/frames.json')
data.head()

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


>#### This dataset contains information describing dialogues related to hotel booking, travel etc.
> - `user_id`: Refers to a unique identifier for the user taking part in the dialogue.
> - `turns`:
>      * `author`: The author of the message in a dialogue. i.e. “user” or “wizard”.
>      * `text`: The sentence that the author uttered. It is the exact text that the author of a turn said. E.g. “text”: “Consider it done. Have a great trip!”.
>      * `labels`: JSON object which has three keys: `active_frame`, `acts`, and `acts_without_refs`.
>           * `active_frame`: The id of the currently active frame.
>           * `acts`: The dialogue acts for the current utterance. Each act has a name and arguments args. The name is the name of the dialogue act, for instance, offer, or inform. The args contain the slot types (key) and slot values (val), for instance budget=$2000. Slot values are optional. An act contains a ref tag whenever a user or wizard refers to a past frame.
>           * `acts_without_refs`: They are similar to the `acts` except that they do not have these ref tags. We define the frame tracking task as the task that takes as input the acts_without_refs and outputs the acts.
>      * `timestamp`: Unix timestamp denoting the time at which the current turn occurred.
>      * `frames`: List of frames up to the current turn. Each frame has the following keys: frame_id, frame_parent_id, requests, binary_questions, compare_requests, and info.
>           * `frame_id`: Id of the frame.
>           * `frame_parent_id`: Id of the parent frame.
>           * `requests, binary_questions, compare_requests`: Requests are questions related to one frame, for instance “what is the price of this package?”. Compare_requests concern several frames. For example, the user might ask to compare different packages: “What is the guest rating of these two hotels?”. Binary_questions are questions with both a slot type and a slot value. These are special cases of requests and compare_requests, for instance “are both hotels 3.5 stars?”.
>           * `info`: The info contains all the constraints set by the user or the wizard in the frame. These constraints are expressed as slot types which have a value. Note that each slot can have multiple values, which accumulate as long as the frame does not change. For example, the price can be both “1000 USD” and “cheapest”. There are two additional fields to keep track of specific aspects of the dialogue: 
>                * REJECTED a boolean value expressing if the user negated or affirmed an offer made by the wizard.
>                * MOREINFO a boolean value expressing whether the user wants to know more about the frame in question
>      * `db`: It can only occur during a wizard’s turn. It is a list of search queries made by the wizard with the associated list of search results. E.g. “db”: {“search”: [{“ORIGIN_CITY”: “Montreal”}], “result”: []}
> - `wizard_id`: Refers to a unique identifier for the wizard taking part in the dialogue.
> - `id`: Refers to a unique identification for the dialogue.
> - `labels`
:
>      * `userSurveyRating`: A value that represents the user’s satisfaction with the Wizard’s service, ranging from 1 – complete dissatisfaction to 5 – complete satisfaction.
>      * `wizardSurveyTaskSuccessful`: A boolean which is true if the wizard thinks at the end of the dialogue that the user’s goal was achieved.

In [218]:
data.describe(include='all').T

,count,unique,top,freq
user_id,1369,11,U22K1SX9N,345
turns,1369,1369,[{'text': 'i wanna go to sacramento from hiros...,1
wizard_id,1369,12,U21T9NMKM,301
id,1369,1369,0ab749ee-6bd0-4560-85c1-59aef778672a,1
labels,1369,16,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc...",929


> **Duplicate**: none
>>
> **Missing**: none
>>
> **Outliers**: not applicable
>>
> **Format errors**: not applicable
>>
> **Lexical errors**: not applicable
>>
> **Multiple Content**: expected


## Let's explore the content of one particular entry

In [221]:
entry_id = 12
x = pd.DataFrame(data['turns'][entry_id])
display(x)

,text,labels,author,timestamp,db
0,Hi im looking for a nice destination that i co...,"{'acts': [{'args': [{'val': 'book', 'key': 'in...",user,1.471277e+12,NaN
1,"I have 2 choices, Vancouver or Toronto.","{'acts': [{'args': [{'val': 'Vancouver', 'key'...",wizard,1.471278e+12,{'result': [[{'trip': {'returning': {'duration...
2,What options are there for both,"{'acts': [{'args': [{'key': 'count_name'}], 'n...",user,1.471278e+12,NaN
3,I have a few. Do you have a budget in mind?,"{'acts': [{'args': [{'key': 'budget'}], 'name'...",wizard,1.471278e+12,"{'result': [], 'search': []}"
4,$3700 is my budget at the moment,"{'acts': [{'args': [{'val': '$3700', 'key': 'b...",user,1.471278e+12,NaN
5,I have the Obsidian Gem Inn in Vancouver which...,"{'acts': [{'args': [{'val': '14 day', 'key': '...",wizard,1.471278e+12,"{'result': [], 'search': []}"
6,Toronto seems like a better place this time of...,"{'acts': [{'args': [{'key': 'dep_time_dst'}, {...",user,1.471278e+12,NaN
7,I have a direct flight to Toronto departing on...,"{'acts': [{'args': [{'val': 'August 15th', 'ke...",wizard,1.471278e+12,"{'result': [], 'search': []}"
8,would by chance have one leaving on the 19th,"{'acts': [{'args': [{'val': '19th', 'key': 'st...",user,1.471278e+12,NaN
9,I have a flight available departing on August ...,"{'acts': [{'args': [{'val': 'August 24th', 'ke...",wizard,1.471278e+12,{'result': [[{'trip': {'returning': {'duration...


### Let's see the complete dialog

In [222]:
for row in x.iterrows():
    time = int(row[1]['timestamp']/1000)
    date_time = datetime.datetime.fromtimestamp(time)
    print(f"{row[1]['author']:8} {date_time.strftime('%Y-%m-%d %H:%M:%S')}: {row[1]['text']}")

user     2016-08-15 18:10:17: Hi im looking for a nice destination that i could go to from Columbus
wizard   2016-08-15 18:12:11: I have 2 choices, Vancouver or Toronto.
user     2016-08-15 18:13:31: What options are there for both
wizard   2016-08-15 18:14:24: I have a few. Do you have a budget in mind?
user     2016-08-15 18:14:49: $3700 is my budget at the moment
wizard   2016-08-15 18:17:51: I have the Obsidian Gem Inn in Vancouver which is a 4-star hotel at $1934.64 or the Hotel Richard in Toronto which is a 3-star hotel at $2393.00. Both are for a 14 day stay.
user     2016-08-15 18:19:06: Toronto seems like a better place this time of year, when would the flights be?
wizard   2016-08-15 18:20:36: I have a direct flight to Toronto departing on August 15th and returning on August 28th.  Would you like me to book that for you?
user     2016-08-15 18:20:54: would by chance have one leaving on the 19th
wizard   2016-08-15 18:24:02: I have a flight available departing on August 24th a

### Let's see the labels associated with the FIRST sentence of the dialog

In [223]:
data['turns'][entry_id][0]

{'text': 'Hi im looking for a nice destination that i could go to from Columbus',
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Columbus', 'key': 'or_city'}], 'name': 'inform'},
   {'args': [], 'name': 'greeting'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Columbus', 'key': 'or_city'}], 'name': 'inform'},
   {'args': [], 'name': 'greeting'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'or_city': [{'val': 'Columbus', 'negated': False}]},
    'frame_id': 1,
    'requests': [],
    'frame_parent_id': None,
    'binary_questions': [],
    'compare_requests': []}]},
 'author': 'user',
 'timestamp': 1471277417489.0}

### Let's see the labels associated with the LAST sentence of the dialog

In [224]:
data['turns'][entry_id][-1]

{'text': 'thank you',
 'labels': {'acts': [{'args': [], 'name': 'thankyou'}],
  'acts_without_refs': [{'args': [], 'name': 'thankyou'}],
  'active_frame': 8,
  'frames': [{'info': {'str_date': [{'val': '19', 'negated': False}],
     'intent': [{'val': 'book', 'negated': False}],
     'or_city': [{'val': 'Columbus', 'negated': False}],
     'count_name': [{'val': 'a few', 'negated': False}],
     'budget': [{'val': '3700.0', 'negated': False}]},
    'frame_id': 1,
    'requests': [],
    'frame_parent_id': None,
    'binary_questions': [],
    'compare_requests': []},
   {'info': {'intent': [{'val': 'book', 'negated': False}],
     'or_city': [{'val': 'Columbus', 'negated': False}],
     'dst_city': [{'val': 'Vancouver', 'negated': False}]},
    'frame_id': 2,
    'requests': [],
    'frame_parent_id': 1,
    'binary_questions': [],
    'compare_requests': []},
   {'info': {'intent': [{'val': 'book', 'negated': False}],
     'or_city': [{'val': 'Columbus', 'negated': False}],
     'dst_

## Let's convert the content of the dataset to match the JSON format expected by LUIS.ai

In [355]:
def extract_entities(acts):
    
    intent = None
    or_city = None
    dst_city = None
    str_date = None
    end_date = None
    budget = None
    n_adults = None
    
    for x in acts:
        z = x['args']
    
        if len(z) == 0:
            break
        
        for y in z:
    
            if 'intent' in y.values() and 'val' in y.keys():
                intent = y['val']
            if 'or_city' in y.values() and 'val' in y.keys():
                or_city =  y['val']
            if 'dst_city' in y.values() and 'val' in y.keys():
                dst_city =  y['val']
            if 'str_date' in y.values() and 'val' in y.keys():
                str_date =  y['val']
            if 'end_date' in y.values() and 'val' in y.keys():
                end_date =  y['val']
            if 'budget' in y.values() and 'val' in y.keys():
                budget =  y['val']
            if 'n_adults' in y.values() and 'val' in y.keys():
                n_adults =  y['val']
    
    return intent, or_city, dst_city, str_date, end_date, budget, n_adults

In [358]:
def convert_to_luis_json(ids):
    
    json_top = []

    for i in ids:

        row = pd.DataFrame(data['turns'][i])
        
        # We will use only the first sentence of the dialog --> iloc[0]
        text = row.iloc[0]['text']
        acts = row.iloc[0]['labels']['acts']

        intent, or_city, dst_city, str_date, end_date, budget, n_adults = extract_entities(acts)

        if intent == "book":  
            intent = "BookFlight"
        elif intent == None:
            if any([or_city, dst_city, str_date, end_date, budget, n_adults]):
                intent = "BookFlight"
            else:
                continue
                
        entry_dict = {}
        entry_dict['text'] = text
        entry_dict['intent'] = intent
        
        entities = []
        for k, v in zip(["or_city", "dst_city", "str_date", "end_date", "budget", "n_adults"], [or_city, dst_city, str_date, end_date, budget, n_adults]) :
            
            if v is None or v == -1 or v == str(-1):
                continue
                
            try:
                v = re.escape(v)
                match=(re.search(v.lower(), text.lower()))
                entity = {"entity":k, "startPos":match.start(), "endPos":match.end()-1}
                entities.append(entity)
            except AttributeError:
                continue

        entry_dict['entities'] = entities

        json_top.append(entry_dict)
        
    return json_top

### Let's try to convert and display the JSON for the first 2 dialogs

In [359]:
xxx = convert_to_luis_json(range(0,2))
print(json.dumps(xxx, indent=2))

[
  {
    "text": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
    "intent": "BookFlight",
    "entities": [
      {
        "entity": "or_city",
        "startPos": 41,
        "endPos": 47
      },
      {
        "entity": "dst_city",
        "startPos": 27,
        "endPos": 34
      },
      {
        "entity": "str_date",
        "startPos": 52,
        "endPos": 76
      },
      {
        "entity": "budget",
        "startPos": 117,
        "endPos": 120
      },
      {
        "entity": "n_adults",
        "startPos": 82,
        "endPos": 82
      }
    ]
  },
  {
    "text": "Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.",
    "intent": "BookFlight",
    "entities": [
      {
        "entity": "or_city",
        "startPos": 44,
        "endPos": 54
      },
      {
        "entity": "dst_city",
        "startPos": 59,
        "endPos": 68
      },
      {
    

### Now let's convert prepare the `Training` and `Validation` datasets

In [360]:
train_size = round(data.shape[0] * 0.9)
valid_size = data.shape[0] - train_size
train_size, valid_size

(1232, 137)

In [361]:
train_json = convert_to_luis_json(range(0,train_size))
valid_json = convert_to_luis_json(range(train_size, train_size+valid_size))
len(train_json), len(valid_json)

(887, 134)

### Let's save the datasets into json files

In [362]:
with open('data/train.json', 'w') as file:
    json.dump(train_json, file)
    
with open('data/valid.json', 'w') as file:
    json.dump(valid_json, file)

## Unfold area...